In [85]:
import pandas as pd
import pyodbc

# Initialize connection parameters

In [86]:
DB = {'servername': 'MSI',
      'source': 'AdventureWorks2019',
      'destination':'DW',
      'user':'user1',
      'pass':'123456'}

# Create connection

In [87]:
#SourceDB
sourceConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['source'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')

#DestinationDB
destConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE='+DB['destination'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')
cursor=destConn.cursor()

In [88]:
df=pd.read_sql('SELECT * FROM Purchasing.ShipMethod',sourceConn,index_col="ShipMethodID")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [89]:
# df = pd.read_sql('SELECT * FROM Purchasing.[PurchaseOrderHeader]',sourceConn)
# df['ModifiedDate'] =pd.to_datetime(df['ModifiedDate']).dt.date
# date=df['ModifiedDate'].drop_duplicates().reset_index(drop=True)
# print(date)

# Extract data from source DB to pandas DataFrames

In [90]:
Tables= [('Purchasing.ShipMethod','ShipMethodID'),('Production.Product','ProductID'),('Purchasing.Vendor','BusinessEntityID'),('Purchasing.PurchaseOrderHeader','PurchaseOrderID'),
         ('Purchasing.PurchaseOrderDetail',['PurchaseOrderID','PurchaseOrderDetailID']), 
         ('Person.Person','BusinessEntityID'),('Sales.SalesTerritory','TerritoryID'),
         ('Person.CountryRegion','CountryRegionCode'),('Sales.SalesOrderHeader','SalesOrderID'),('Sales.SalesOrderDetail',['SalesOrderID','SalesOrderDetailID'])
        ]

In [91]:
# df=pd.read_sql('SELECT BusinessEntityID FROM HumanResources.Employee',sourceConn,index_col='BusinessEntityID')
# df


In [92]:
dfs={}
for item in Tables:
    dfs[item[0]]=pd.read_sql(f'SELECT * FROM {item[0]}',sourceConn,index_col=item[1])
dfs['HumanResources.Employee']=pd.read_sql('SELECT BusinessEntityID FROM HumanResources.Employee',sourceConn,index_col='BusinessEntityID')


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas

# Transform staged data

## Extract essential columns for data warehouse

In [93]:
#columns to extract in each table
Columns= [('Purchasing.ShipMethod','Name'),('Production.Product',['Name','ProductLine']),('Purchasing.Vendor',['Name','CreditRating']),
         ('Purchasing.PurchaseOrderHeader',['Status','EmployeeID','VendorID','ShipMethodID','OrderDate','ShipDate']),
         ('Purchasing.PurchaseOrderDetail',['DueDate','OrderQty','ProductID','LineTotal','ReceivedQty','RejectedQty','StockedQty']), 
         ('Person.Person',['FirstName','MiddleName','LastName','Suffix']),('Sales.SalesTerritory',['Name','CountryRegionCode','Group']),
         ('Person.CountryRegion','Name'),
         ('Sales.SalesOrderHeader',['OrderDate','DueDate','ShipDate','Status','OnlineOrderFlag','TerritoryID','ShipMethodID']),
         ('Sales.SalesOrderDetail',['OrderQty','ProductID','LineTotal'])
        ]

In [94]:
# df=pd.DataFrame(dfs['Production.Product'][['Name','Color','Size','Class','Style']])
# df

In [95]:
dfst={}
for item in Columns:
    dfst[item[0]]=dfs[item[0]][item[1]]
dfst['HumanResources.Employee']=dfs['HumanResources.Employee']

## Transform  Data for Dimensions

### DimEmpolyee

In [96]:
dfst['Person.Person']['Name']=dfst['Person.Person']['FirstName'].map(str)+' '+dfst['Person.Person']['MiddleName'].map(str)+'. '+dfst['Person.Person']['LastName'].map(str)
dfst['HumanResources.Employee']=pd.merge(dfst['HumanResources.Employee'],dfst['Person.Person'],how='left',on='BusinessEntityID')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_25564\1021075203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfst['Person.Person']['Name']=dfst['Person.Person']['FirstName'].map(str)+' '+dfst['Person.Person']['MiddleName'].map(str)+'. '+dfst['Person.Person']['LastName'].map(str)


### DimProduct

In [97]:
mapper = {'R':'Road','M':'Mountain','T':'Touring','S':'Standard'}
dfst['Production.Product']['ProductLine'].replace(mapper)


ProductID
1      None
2      None
3      None
4      None
316    None
       ... 
995    None
996    None
997      R 
998      R 
999      R 
Name: ProductLine, Length: 504, dtype: object

In [79]:
dfst['Production.Product']['ProductLine']=dfst['Production.Product']['ProductLine'].to_string()
dfst['Production.Product']
# dfst['Production.Product']['ProductLine'].map({'R':'Road','M':'Mountain','T':'Touring','S':'Standard'})

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_25564\1238263580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfst['Production.Product']['ProductLine']=dfst['Production.Product']['ProductLine'].to_string()


,Name,ProductLine
ProductID,,
1,Adjustable Race,ProductID\n1 None\n2 None\n3 No...
2,Bearing Ball,ProductID\n1 None\n2 None\n3 No...
3,BB Ball Bearing,ProductID\n1 None\n2 None\n3 No...
4,Headset Ball Bearings,ProductID\n1 None\n2 None\n3 No...
316,Blade,ProductID\n1 None\n2 None\n3 No...
...,...,...
995,ML Bottom Bracket,ProductID\n1 None\n2 None\n3 No...
996,HL Bottom Bracket,ProductID\n1 None\n2 None\n3 No...
997,"Road-750 Black, 44",ProductID\n1 None\n2 None\n3 No...


In [82]:
dfst['Production.Product']['ProductLine'].str.replace(r'^R$','Road',regex=True)
# dfst['Production.Product']['ProductLine']=dfst['Production.Product']['ProductLine'].str.replace(r'^M$','Mountain',regex=True)
# dfst['Production.Product']['ProductLine']=dfst['Production.Product']['ProductLine'].str.replace(r'^T$','Touring',regex=True)
# dfst['Production.Product']['ProductLine']=dfst['Production.Product']['ProductLine'].str.replace(r'^S$','Standard',regex=True)
dfst['Production.Product']

,Name,ProductLine
ProductID,,
1,Adjustable Race,ProductID\n1 None\n2 None\n3 No...
2,Bearing Ball,ProductID\n1 None\n2 None\n3 No...
3,BB Ball Bearing,ProductID\n1 None\n2 None\n3 No...
4,Headset Ball Bearings,ProductID\n1 None\n2 None\n3 No...
316,Blade,ProductID\n1 None\n2 None\n3 No...
...,...,...
995,ML Bottom Bracket,ProductID\n1 None\n2 None\n3 No...
996,HL Bottom Bracket,ProductID\n1 None\n2 None\n3 No...
997,"Road-750 Black, 44",ProductID\n1 None\n2 None\n3 No...


In [24]:
for index, row in df.iterrows():
      cursor.execute('''INSERT INTO dbo.DimEmployee (EmployeeID,Name) values(?,?)''',index, row.Name)      
cursor.commit()
